## Set up

In [5]:
import numpy as np
import os.path as op
from pprint import pformat
from typing import Tuple, Iterator
import time

# EEG utilities
import mne
from mne.preprocessing import ICA, create_eog_epochs
from pyprep.prep_pipeline import PrepPipeline
from autoreject import get_rejection_threshold, validation_curve

# BIDS utilities
from mne_bids import BIDSPath, read_raw_bids
from util.io.bids import DataSink
from bids import BIDSLayout

In [6]:
# Constants
BIDS_ROOT = '../data/bids'
DERIV_ROOT = op.join(BIDS_ROOT, 'derivatives')
CHANMAP_FPATH = '../data/captrak/CACS-64_NO_REF.bvef'
CHANPOS_ROOT = '../data/captrak'
LOWPASS = 300
FS = 2000
REJECT_THRES = 5e-7 # 50 microvolts

## Functions
#### Import data 

In [7]:
# Create Iterator object to loop over all files
KeyType = Tuple[str, str, str]

def fpaths() -> Iterator[KeyType]:
    for sub in subjects:
        for task in tasks:
            for run in runs:
                run = str(run) # layout.get_runs() doesn't return strings for some reason
                key = (sub, task, run)
                yield key

def get_bids_path(bids_root, sub, task, run):
    bids_path = BIDSPath(root = bids_root,
                        subject = sub,
                        task = task,
                        run = run,
                        datatype = 'eeg',
                        )
    return bids_path

def import_bids_data(bids_path):
    raw = read_raw_bids(bids_path, verbose = False)
    raw = raw.pick_types(eeg = True)
    return raw

def read_events(raw):
    events, events_ids = mne.events_from_annotations(raw)
    return events, events_ids

#### Set electrode locations and create EOGs

In [23]:
def set_electrode_mappings(chanmap_fpath):
    dig = mne.channels.read_custom_montage(chanmap_fpath)
    mapping = {'Ch%s'%i: dig.ch_names[i] for i in range(len(dig.ch_names))}
#     del mapping['Ch33']
#     del mapping['Ch61']
#     del mapping['Ch64']
    return mapping

def get_chanpos_fpath(chanpos_root, sub):
    chanpos_fpath = op.join(chanpos_root, 'subj_' + sub + '.bvct')
    return chanpos_fpath

def set_electrode_positions(raw, chanpos_fpath, stim_channel, mapping):
    dig = mne.channels.read_dig_captrak(chanpos_fpath)
    raw = raw.set_channel_types({stim_channel: 'stim'}) 
    raw = raw.rename_channels(mapping)
    raw = raw.set_montage(dig)
    return raw

def create_eogs(raw):
    raw = mne.set_bipolar_reference(raw, anode = 'Fp1', cathode = 'AF7', ch_name = 'eog1', drop_refs = False)
    raw = mne.set_bipolar_reference(raw, anode = 'Fp2', cathode = 'AF8', ch_name = 'eog2', drop_refs = False)
    raw = raw.set_channel_types({'eog1': 'eog', 'eog2': 'eog'})
    return raw

#### Resampling and PREP

In [9]:
def resample(raw, fs, events): # Resample to a more manageable speed
    raw, events = raw.resample(fs, events = events)
    return raw, events

def run_PREP(raw, sub, run, LOWPASS): # Run PREP pipeline (notch, exclude bad channels, and re-reference)
    raw.load_data()
    seed = int(str(sub) + str(run))
    np.random.seed(seed)

    lf = raw.info['line_freq']
    prep_params = {
        'ref_chs': 'eeg',
        'reref_chs': 'eeg',
        'line_freqs': np.arange(lf, LOWPASS, lf) if np.arange(lf, LOWPASS, lf).size > 0 else [lf]
    }
    prep = PrepPipeline(raw, prep_params, raw.get_montage(), ransac = False, random_state = seed)
    prep = prep.fit()

    raw = prep.raw_eeg # replace raw with cleaned version
    bads = prep.noisy_channels_original
    return raw, bads

#### Run ICA on one copy of the data
Split the data into two copies, one filtered more liberally for ICA so that high frequency noise can be detected, one band-pass filtered at the behaviorally relevant frequencies. All of the following preprocessing steps will be applied to each of the copies.

In [10]:
def bandpass(raw, h_freq, l_freq):
    raw = raw.filter(l_freq = l_freq, h_freq = h_freq)
    return raw

def epoch(raw):
    epochs = mne.Epochs(
        raw, 
        events, 
        tmin = -0.2, 
        tmax = 0.250, 
        baseline = None, # do NOT baseline correct the trials yet; we do that after ICA
        event_id = event_ids, # remember which epochs are associated with which condition
        preload = True # keep data in memory
    )
    return epochs

def compute_ICA(epochs):
    ica = ICA(n_components = 15, random_state = 0)
    ica = ica.fit(epochs, picks = 'eeg')
    return ica

def apply_ICA(epochs_for_ica, epochs):
    eog_indices, eog_scores = ica.find_bads_eog(epochs_for_ica, threshold = 1.96)
    ica.exclude = eog_indices
    epochs = ica.apply(epochs) # apply to aggressively filtered version of data
    return epochs, ica

#### Baseline correct and reject trials
Back to applying preprocessing on only one copy of the data. ICA is finished.

In [11]:
def baseline_correct(epochs):
    epochs = epochs.pick_types(eeg = True) # change syntax?
    epochs = epochs.apply_baseline((-0.2, 0.))
    return epochs

def reject_trials(threshold, epochs):
    epochs = epochs.drop_bad(reject = {'eeg': threshold})
    return epochs

#### Save results and generate report

In [12]:
def get_save_path(deriv_root, sub, task, run):
    sink = DataSink(folder, 'preprocessing')

    # save cleaned data
    fpath = sink.get_path(
                    subject = sub,
                    task = task, 
                    run = run,
                    desc = 'clean',
                    suffix = 'epo', # this suffix is following MNE, not BIDS, naming conventions
                    extension = 'fif.gz',
                    )
    return fpath

def save_preprocessed_data(fpath, epochs):
    epochs.save(fpath, overwrite = True)
    
def generate_report(fpath, epochs, ica, bads):
    report = mne.Report(verbose = True)
    report.parse_folder(op.dirname(fpath), pattern = '*epo.fif.gz', render_bem = False)

    # Plot the ERP
    fig_erp = epochs['50'].average().plot(spatial_colors = True)
    report.add_figs_to_section(
        fig_erp, 
        captions = 'Average Evoked Response', 
        section = 'evoked'
    )

    # Plot the excluded ICAs
    if ica.exclude: # if we found any bad components
        fig_ica_removed = ica.plot_components(ica.exclude)
        report.add_figs_to_section(
            fig_ica_removed, 
            captions = 'Removed ICA Components', 
            section = 'ICA'
        )     
    
    # Format output
    html_lines = []
    for line in pformat(bads).splitlines():
        html_lines.append('<br/>%s' % line) 
    html = '\n'.join(html_lines)
    report.add_htmls_to_section(html, captions = 'Interpolated Channels', section = 'channels')
    report.add_htmls_to_section('<br/>threshold: {:0.2f} microvolts</br>'.format(thres['eeg'] * 1e6), 
                                captions = 'Trial Rejection Criteria', section = 'rejection')
    report.add_htmls_to_section(epochs.info._repr_html_(), captions = 'Info', section = 'info')
    report.save(op.join(sink.deriv_root, 'sub-%s.html'%sub), overwrite = True)

## Preprocessing wrapper
Since we have to loop over all the data files the section below will contain the for loop to wrap all the preprocessing functions contained in the subsequent sections.

In [13]:
# start = time.time()

# # Parse BIDS directory
# layout = BIDSLayout(BIDS_ROOT)
# subjects = layout.get_subjects()
# tasks = layout.get_tasks()
# runs = layout.get_runs()
# print(subjects, tasks, runs)

# for (sub, task, run) in fpaths():
    
#     # Import data
#     print("Import data")
#     bids_path = get_bids_path(BIDS_ROOT, sub, task, run)
#     print(bids_path)
#     if not bids_path.fpath.is_file(): # skip if file doesn't exist
#         continue
#     raw = import_bids_data(bids_path)
#     events, event_ids = read_events(raw)
    
#     # Set channel locations and create EOGs
#     mapping = set_electrode_mappings(CHANMAP_FPATH)
#     chanpos_fpath = get_chanpos_fpath(CHANPOS_ROOT, sub)
#     raw = set_electrode_positions(raw, chanpos_fpath, 'Aux1', mapping)
#     raw = create_eogs(raw)
    
#     # Resampling and PREP
#     print("Resampling and PREP")
#     raw, events = resample(raw, FS, events)
#     raw, bads = run_PREP(raw, sub, run, LOWPASS)
    
#     # Run ICA on one copy of the data
#     print("Run ICA on one copy of the data")
#     raw_for_ica = bandpass(raw, 1., None)
#     raw = bandpass(raw, 30, 270)
    
#     epochs_for_ica = epoch(raw_for_ica)
#     epochs = epoch(raw)
    
#     ica = compute_ICA(epochs_for_ica) # run ICA on less aggressively filtered data
#     epochs, ica = apply_ICA(epochs_for_ica, epochs) # apply ICA on more aggressively filtered data
    
#     # Baseline correct and reject trials
#     print("Baseline correct and reject trials")
#     epochs = baseline_correct(epochs)
#     epochs = reject_trials(REJECT_THRES, epochs)
    
#     # Save results and generate report
#     print("Save results and generate report")
#     fpath = get_save_path(DERIV_ROOT, sub, task, run)
#     save_preprocessed_data(fpath, epochs)
#     generate_report(fpath, epochs, ica, bads)
    
# end = time.time()
# print(end - start)

In [28]:
start = time.time()

sub = '2'
task = 'pitch'
run = '1'

# Import data
print("Import data")
bids_path = get_bids_path(BIDS_ROOT, sub, task, run)
print(bids_path)
raw = import_bids_data(bids_path)
raw.ch_names

# # Set channel locations and create EOGs
# mapping = set_electrode_mappings(CHANMAP_FPATH)
# chanpos_fpath = get_chanpos_fpath(CHANPOS_ROOT, sub)
# raw = set_electrode_positions(raw, chanpos_fpath, 'Aux1', mapping)
# raw = create_eogs(raw)

# end = time.time()
# print(end - start)

Import data
../data/bids/sub-2/eeg/sub-2_task-pitch_run-1_eeg.vhdr


['Fp1',
 'Fz',
 'F3',
 'F7',
 'FT9',
 'FC5',
 'FC1',
 'C3',
 'T7',
 'TP9',
 'CP5',
 'CP1',
 'Pz',
 'P3',
 'P7',
 'O1',
 'Oz',
 'O2',
 'P4',
 'P8',
 'TP10',
 'CP6',
 'CP2',
 'C4',
 'T8',
 'FT10',
 'FC6',
 'FC2',
 'F4',
 'F8',
 'Fp2',
 'AF7',
 'AF3',
 'AFz',
 'F1',
 'F5',
 'FT7',
 'FC3',
 'C1',
 'C5',
 'TP7',
 'CP3',
 'P1',
 'P5',
 'PO7',
 'PO3',
 'POz',
 'PO4',
 'PO8',
 'P6',
 'P2',
 'CPz',
 'CP4',
 'TP8',
 'C6',
 'C2',
 'FC4',
 'FT8',
 'F6',
 'AF8',
 'AF4',
 'F2',
 'FCz',
 'Aux1']

In [21]:
raw.ch_names

['Fp1',
 'Fz',
 'F3',
 'F7',
 'FT9',
 'FC5',
 'FC1',
 'C3',
 'T7',
 'TP9',
 'CP5',
 'CP1',
 'Pz',
 'P3',
 'P7',
 'O1',
 'Oz',
 'O2',
 'P4',
 'P8',
 'TP10',
 'CP6',
 'CP2',
 'C4',
 'T8',
 'FT10',
 'FC6',
 'FC2',
 'F4',
 'F8',
 'Fp2',
 'AF7',
 'AF3',
 'AFz',
 'F1',
 'F5',
 'FT7',
 'FC3',
 'C1',
 'C5',
 'TP7',
 'CP3',
 'P1',
 'P5',
 'PO7',
 'PO3',
 'POz',
 'PO4',
 'PO8',
 'P6',
 'P2',
 'CPz',
 'CP4',
 'TP8',
 'C6',
 'C2',
 'FC4',
 'FT8',
 'F6',
 'AF8',
 'AF4',
 'F2',
 'FCz',
 'Aux1']

In [22]:
mapping

{'Ch0': 'GND',
 'Ch1': 'Fp1',
 'Ch2': 'Fz',
 'Ch3': 'F3',
 'Ch4': 'F7',
 'Ch5': 'FT9',
 'Ch6': 'FC5',
 'Ch7': 'FC1',
 'Ch8': 'C3',
 'Ch9': 'T7',
 'Ch10': 'TP9',
 'Ch11': 'CP5',
 'Ch12': 'CP1',
 'Ch13': 'Pz',
 'Ch14': 'P3',
 'Ch15': 'P7',
 'Ch16': 'O1',
 'Ch17': 'Oz',
 'Ch18': 'O2',
 'Ch19': 'P4',
 'Ch20': 'P8',
 'Ch21': 'TP10',
 'Ch22': 'CP6',
 'Ch23': 'CP2',
 'Ch24': 'Cz',
 'Ch25': 'C4',
 'Ch26': 'T8',
 'Ch27': 'FT10',
 'Ch28': 'FC6',
 'Ch29': 'FC2',
 'Ch30': 'F4',
 'Ch31': 'F8',
 'Ch32': 'Fp2',
 'Ch34': 'AF3',
 'Ch35': 'AFz',
 'Ch36': 'F1',
 'Ch37': 'F5',
 'Ch38': 'FT7',
 'Ch39': 'FC3',
 'Ch40': 'C1',
 'Ch41': 'C5',
 'Ch42': 'TP7',
 'Ch43': 'CP3',
 'Ch44': 'P1',
 'Ch45': 'P5',
 'Ch46': 'PO7',
 'Ch47': 'PO3',
 'Ch48': 'POz',
 'Ch49': 'PO4',
 'Ch50': 'PO8',
 'Ch51': 'P6',
 'Ch52': 'P2',
 'Ch53': 'CPz',
 'Ch54': 'CP4',
 'Ch55': 'TP8',
 'Ch56': 'C6',
 'Ch57': 'C2',
 'Ch58': 'FC4',
 'Ch59': 'FT8',
 'Ch60': 'F6',
 'Ch62': 'AF4',
 'Ch63': 'F2'}